In [1]:
import pandas as pd
import numpy as np
import time
import json
import requests
from datetime import date
from matplotlib import pyplot as plt
import datetime

In [2]:
# RANDOMLY SELECT AN API KEY AND USE IT FOR THE API CALLS
api_keys = ['0a9d962918e369aa52bf64266bf11f57',
            'e76fcf746d3ca3cc025c0803dd212fc8']

In [3]:
county_level_feat = pd.read_csv('county_features_final.csv')

In [4]:
chosen_county_feat = np.unique(county_level_feat.title.values)
print(np.unique(county_level_feat['frequency']))
print(len(chosen_county_feat))
print(chosen_county_feat)

['Annual' 'Monthly' 'Quarterly']
73
['90% Confidence Interval Lower Bound of Estimate of Median Household Income'
 '90% Confidence Interval Lower Bound of Estimate of People Age 0-17 in Poverty'
 '90% Confidence Interval Lower Bound of Estimate of People of All Ages in Poverty'
 '90% Confidence Interval Lower Bound of Estimate of Percent of People Age 0-17 in Poverty'
 '90% Confidence Interval Lower Bound of Estimate of Percent of People of All Ages in Poverty'
 '90% Confidence Interval Lower Bound of Estimate of Percent of Related Children Age 5-17 in Families in Poverty'
 '90% Confidence Interval Lower Bound of Estimate of Related Children Age 5-17 in Families in Poverty'
 '90% Confidence Interval Upper Bound of Estimate of Median Household Income'
 '90% Confidence Interval Upper Bound of Estimate of People Age 0-17 in Poverty'
 '90% Confidence Interval Upper Bound of Estimate of People of All Ages in Poverty'
 '90% Confidence Interval Upper Bound of Estimate of Percent of People Age

In [5]:
county_level_feat.head()

,Unnamed: 0,frequency,id,observation_end,observation_start,seasonal_adjustment,title,units,county_id,num_months
0,0,Annual,2020RATIO001001,2018-01-01,2010-01-01,Not Seasonally Adjusted,Income Inequality,Ratio,27336,96
1,1,Monthly,ALAUTA1LFN,2020-02-01,1990-01-01,Not Seasonally Adjusted,Civilian Labor Force,Persons,27336,361
2,2,Annual,ALAUTA1POP,2019-01-01,1970-01-01,Not Seasonally Adjusted,Resident Population,Thousands of Persons,27336,588
3,3,Monthly,ALAUTA1URN,2020-02-01,1990-01-01,Not Seasonally Adjusted,Unemployment Rate,Percent,27336,361
4,4,Annual,ATNHPIUS01001A,2018-01-01,1986-01-01,Not Seasonally Adjusted,All-Transactions House Price Index,Index 2000=100,27336,384


In [6]:
county_level_feat = county_level_feat.sort_values(["county_id", "id"])

In [8]:
counties_list = np.unique(county_level_feat['county_id'].values)
feat_ids = county_level_feat['id'].values

time_freq = county_level_feat['frequency'].values
df_series_table = pd.DataFrame()
series_table_cols = ['id', 'title', 'observation_start', 'observation_end', 'frequency', 'units', 'seasonal_adjustment']
start = 0
end = 7
count = 0

failed_ids = []
for county_id in counties_list[start:end]:
    data = None

    try:
        time.sleep(1)
        params = {
            'category_id': county_id,
            'api_key': api_keys[0],
            'file_type': 'json'
        }
        r = requests.get(url = 'https://api.stlouisfed.org/fred/category/series', params=params)
        res = r.json()
        # Get the observations for a series
        seriess = res['seriess']
        res2 = []
        for series in range(len(seriess)):
            current_series = seriess[series]
            series_id = current_series['id']
            freq = current_series['frequency']

            params = {
                'series_id': series_id,
                'api_key': api_keys[0],
                'file_type': 'json'
            }
            r = requests.get(url = 'https://api.stlouisfed.org/fred/series/observations', params=params)

            res1 = r.json()
            res1["observations"] = [i for i in res1["observations"] if not (int(i["date"][0:4]) < 2010 or int(i["date"][0:4]) > 2017)]
                
            for val in res1["observations"]:
                del val["realtime_start"]
                del val["realtime_end"]

            if series_id in feat_ids:
                for val in res1['observations']:
                    if (freq == "Annual"):
                        val['feature'] = series_id
                        res2 += [val]
                        for i in range(2, 13):
                            if i < 10:
                                string = '0' + str(i)
                            else:
                                string = str(i)
                            dict_to_add = {}
                            dict_to_add['feature'] = series_id
                            dict_to_add['date'] = val['date'][0:5] + string + val['date'][7:10]
                            dict_to_add['value'] = val['value']
                            res2 += [dict_to_add]
                    elif (freq == "Quarterly"):
                        val['feature'] = series_id
                        res2 += [val]
                        for i in range(1, 3):
                            cur_date = int(val['date'][5:7])
                            date = i + cur_date
                            if date < 10:
                                date = '0' + str(date)
                            else:
                                date = str(date)

                            dict_to_add = {}
                            dict_to_add['feature'] = series_id
                            dict_to_add['date'] = val['date'][0:5] + date + val['date'][7:10]
                            dict_to_add['value'] = val['value']
                            res2 += [dict_to_add]
                    else: #monthly
                        val['feature'] = series_id
                        res2 += [val]

        dates = []
        for i in range(96):
            dates += [res2[i]['date']]

        res3 = {}
        for val in res2:
            if val['feature'] not in res3.keys():
                # new thing
                res3[val['feature']] = [val['value']]
            else:
                res3[val['feature']] += [val['value']]

        for feat in res3:
            if len(res3[feat]) != 96:
                del res3[feat]

        name = str(county_id) + '.csv'

        data = pd.DataFrame(res3, index=dates)
        data.to_csv(name)
        print('csv written:', county_id)
    except:
        print('County ID Failed, rate limited:', county_id)
        failed_ids += [county_id]
print('Failed counties:')
for i in failed_ids:
    print(i)

csv written: 204
csv written: 205
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
County ID Failed, rate limited: 206
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
County ID Failed, rate limited: 207
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
County ID Failed, rate limited: 209
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
{'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
County ID Failed, rate limited: 210
County ID Failed, rate limited: 211
Failed counties:
206
207
209
210
211
